In [1]:
from dotenv import load_dotenv
import os
import sys
sys.path.append("..")

In [10]:
from database.qdrant_client import QdrantClient
from tqdm import tqdm
from qdrant_client.models import Distance, VectorParams, PointStruct

In [3]:
load_dotenv()

True

In [6]:
qdrant_client = QdrantClient()

2025-04-16 21:30:15,593 | INFO | sentence_transformers.SentenceTransformer | Use pytorch device_name: cuda:0
2025-04-16 21:30:15,595 | INFO | sentence_transformers.SentenceTransformer | Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-04-16 21:30:27,222 | INFO | httpx | HTTP Request: GET http://172.26.33.199:2513 "HTTP/1.1 200 OK"


In [8]:
with open("../assets/ontologies/CHeVIE_comment.owl", "r") as f:
    data = f.read()

In [9]:
code_parts = data.split("\n\n\n")

In [11]:
points = []

for idx, code_part in tqdm(enumerate(code_parts)):
    embedding = qdrant_client.default_model.encode(code_part)
    point = {
        "id": idx,
        "vector": embedding.tolist(),
        "payload": {"code": code_part}
    }
    points.append(point)

In [ ]:
qdrant_client.upsert_points(
    "ontology_embedding",
    points
)

In [61]:
search_result = qdrant_client.client.query_points(
    collection_name="ontology_embedding", 
    query=qdrant_client.default_model.encode(
        "Tìm thời gian bắt đầu của triều đại Triệu?"
    ), limit=3
).points

Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 34.71it/s]

2025-04-17 02:56:19,949 | INFO | httpx | HTTP Request: POST http://172.26.33.199:2513/collections/ontology_embedding/points/query "HTTP/1.1 200 OK"


In [62]:
code = ""
for res in search_result:
    code += res.payload["code"].strip() 

In [63]:
code.strip()

'<!-- https://CHeVIE.vn/ontologies/TrieuRulingDynastyStartDate -->\n\n    <owl:NamedIndividual rdf:about="https://CHeVIE.vn/ontologies/TrieuRulingDynastyStartDate">\n        <rdf:type rdf:resource="http://www.w3.org/2006/time#Instant"/>\n        <time:inDateTime rdf:resource="https://CHeVIE.vn/ontologies/TrieuRulingDynastyStartDateDescription"/>\n        <rdfs:label xml:lang="en">Start Date of Triệu Ruling Dynasty</rdfs:label>\n    \t<rdfs:label xml:lang="vi">Ngày bắt đầu triều đại Triệu</rdfs:label>\n    </owl:NamedIndividual><!-- https://CHeVIE.vn/ontologies/TrieuRulingDynastyStartDateDescription -->\n\n    <owl:NamedIndividual rdf:about="https://CHeVIE.vn/ontologies/TrieuRulingDynastyStartDateDescription">\n        <rdf:type rdf:resource="http://www.w3.org/2006/time#DateTimeDescription"/>\n        <time:hasTRS rdf:resource="https://CHeVIE.vn/ontologies/GregorianCalendar"/>\n        <time:year rdf:datatype="http://www.w3.org/2001/XMLSchema#gYear">-210</time:year>\n        <rdfs:label